In [17]:
# Import necessary modules
import os
import mysql.connector as connector  # MySQL connector for database interaction
from mysql.connector import errorcode  # Import error codes for exception handling
import pandas as pd
from IPython.display import display  # For displaying DataFrame in Jupyter Notebook

# Load MySQL Credentials from Environment Variables
DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")
DB_NAME = os.getenv("DB_NAME")
DB_HOST = os.getenv("DB_HOST", "localhost")  # Default to localhost if not specified

# Establish MySQL Connection
try:
    connection = connector.connect(
        user=DB_USER,  
        password=DB_PASSWORD,  
        database=DB_NAME,
        host=DB_HOST
    )
    print("\nConnected to MySQL database")
except connector.Error as err:
    if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
        print("Connection failed: Incorrect username or password")
    elif err.errno == errorcode.ER_BAD_DB_ERROR:
        print("Database does not exist")
    else:
        print(f"Error: {err}")
    exit()  # Exit script if connection fails

# Create cursor
cursor = connection.cursor()    


Connected to MySQL database


In [19]:
# Retrieve the newly added bookings for 2025-03-20
bookings_query = """
SELECT * 
FROM Bookings 
WHERE BookingDate = "2025-03-20";
"""

cursor.execute(bookings_query)
df_bookings = pd.DataFrame(cursor.fetchall(), columns=[desc[0] for desc in cursor.description])

# Display the bookings for 2025-03-20
print("\n📌 Bookings for 2025-03-20:")
display(df_bookings)


📌 Bookings for 2025-03-20:


,BookingID,CustomerID,BookingDate,TableID,Status
0,195,10,2025-03-20,2,New Booking
1,196,11,2025-03-20,4,New Booking
2,197,12,2025-03-20,6,New Booking
3,198,13,2025-03-20,8,New Booking
4,199,14,2025-03-20,10,New Booking


In [20]:
# Define the staff ID
STAFF_ID = 20  # Select staff member

# Track created orders
created_orders = []

# Create an order for each booking
try:
    for booking in bookings:
        booking_id, customer_id = booking  # Unpack tuple

        # Call the stored procedure CreateOrder
        cursor.callproc('CreateOrder', (booking_id, customer_id, STAFF_ID, "Admin"))

        # Fetch the result (New OrderID)
        for result in cursor.stored_results():
            new_order_id = result.fetchone()[0]  # Extract OrderID
            created_orders.append((new_order_id, booking_id, customer_id))
    
    # Commit all transactions
    connection.commit()
    print("\n✅ Orders successfully created for all bookings on 2025-03-20.")

except connector.Error as err:
    print(f"❌ Error creating orders: {err}")

# Convert to DataFrame and display
df_orders = pd.DataFrame(created_orders, columns=["OrderID", "BookingID", "CustomerID"])
display(df_orders)


✅ Orders successfully created for all bookings on 2025-03-20.


,OrderID,BookingID,CustomerID
0,190,195,10
1,191,196,11
2,192,197,12
3,193,198,13
4,194,199,14


In [24]:
# Retrieve all orders from the Orders table
orders_query = """
SELECT * FROM Orders
WHERE OrderDate = "2025-03-21";
"""

# Execute the query
cursor.execute(orders_query)

# Fetch all records and convert to Pandas DataFrame
df_orders = pd.DataFrame(cursor.fetchall(), columns=[desc[0] for desc in cursor.description])

# Display the orders
print("\n📌 Orders in the Orders Table for 2025-03-21:")
display(df_orders)


📌 Orders in the Orders Table for 2025-03-21:


,OrderID,CustomerID,StaffID,OrderDate,TotalCost,BookingID
0,190,10,20,2025-03-21,0.00,195
1,191,11,20,2025-03-21,0.00,196
2,192,12,20,2025-03-21,0.00,197
3,193,13,20,2025-03-21,0.00,198
4,194,14,20,2025-03-21,0.00,199


In [25]:
# Define test menu items and quantities for each order
menu_selections = {
    190: ("11,12,14", "2,1,3"),  # Order 190 -> Spaghetti Bolognese, Caesar Salad, Fried Chicken
    191: ("15,16,18", "1,1,2"),  # Order 191 -> Margarita Pizza, Penne Arrabbiata, Grilled Salmon
    192: ("20,21,22", "3,2,2"),  # Order 192 -> Iced Tea, Coke, Beer
    193: ("24,26,28", "1,2,1"),  # Order 193 -> Red Wine, Chocolate Cake, Tiramisu
    194: ("32,33,35", "2,1,1")   # Order 194 -> Hot Chocolate, Fried Fish, Vegetable Stir Fry
}

# Add items to each order
try:
    for order_id, (menu_ids, quantities) in menu_selections.items():
        cursor.callproc('AddMultipleItemsToOrder', (order_id, menu_ids, quantities))
    
    # Commit transaction
    connection.commit()
    print("\nMenu items successfully added to orders.")

except connector.Error as err:
    print(f"Error adding items to orders: {err}")

# Retrieve and display the updated order items
order_items_query = """
SELECT * FROM Order_Items;
"""

cursor.execute(order_items_query)
df_order_items = pd.DataFrame(cursor.fetchall(), columns=[desc[0] for desc in cursor.description])

print("\n📌 Order Items After Adding Menu Items:")
display(df_order_items)


Menu items successfully added to orders.

📌 Order Items After Adding Menu Items:


,OrderItemID,OrderID,MenuID,Quantity,ItemPrice,TotalItemCost
0,135,153,12,2,10.00,20.00
1,136,153,14,1,14.00,14.00
2,137,153,18,3,22.00,66.00
3,138,154,22,1,5.00,5.00
4,139,154,26,2,6.00,12.00
...,...,...,...,...,...,...
70,244,193,26,2,6.00,12.00
71,245,193,28,1,6.00,6.00
72,246,194,32,2,3.50,7.00
73,247,194,33,1,3.75,3.75


In [26]:
# Retrieve all orders from the Orders table
orders_query = """
SELECT * FROM Orders
WHERE OrderDate = "2025-03-21";
"""

# Execute the query
cursor.execute(orders_query)

# Fetch all records and convert to Pandas DataFrame
df_orders = pd.DataFrame(cursor.fetchall(), columns=[desc[0] for desc in cursor.description])

# Display the orders
print("\n📌 Orders in the Orders Table for 2025-03-21:")
display(df_orders)


📌 Orders in the Orders Table for 2025-03-21:


,OrderID,CustomerID,StaffID,OrderDate,TotalCost,BookingID
0,190,10,20,2025-03-21,82.00,195
1,191,11,20,2025-03-21,69.50,196
2,192,12,20,2025-03-21,25.50,197
3,193,13,20,2025-03-21,25.50,198
4,194,14,20,2025-03-21,24.25,199


In [27]:
order_status_query = """
SELECT 
    o.OrderID,
    b.BookingID,
    b.CustomerID,
    CONCAT(c.FirstName, ' ', c.LastName) AS CustomerName,
    o.OrderDate,
    o.StaffID,
    CONCAT(s.FirstName, ' ', s.LastName) AS StaffMember,
    ods.Status AS OrderStatus,
    b.Status AS BookingStatus
FROM Orders o
LEFT JOIN Bookings b ON o.BookingID = b.BookingID
LEFT JOIN Customer_Details c ON b.CustomerID = c.CustomerID
LEFT JOIN Staff_Information s ON o.StaffID = s.StaffID
LEFT JOIN Order_Delivery_Status ods ON o.OrderID = ods.OrderID;
"""

# Execute query
cursor.execute(order_status_query)

# Fetch results and convert them into a DataFrame
df = pd.DataFrame(cursor.fetchall(), columns=[desc[0] for desc in cursor.description])

# Display DataFrame in Jupyter Notebook
display(df)  # Using Pandas display function

,OrderID,BookingID,CustomerID,CustomerName,OrderDate,StaffID,StaffMember,OrderStatus,BookingStatus
0,152,NaN,NaN,None,2025-03-14,7,John Doe,Cancelled,None
1,153,146.0,10.0,Jane Doe,2025-03-15,7,John Doe,Delivered,New Booking
2,154,147.0,11.0,Bob Smith,2025-03-15,7,John Doe,Delivered,New Booking
3,155,148.0,12.0,Alice Johnson,2025-03-15,7,John Doe,Delivered,New Booking
4,156,149.0,13.0,Charlie Brown,2025-03-15,7,John Doe,Delivered,New Booking
5,157,150.0,14.0,David Taylor,2025-03-15,7,John Doe,Delivered,New Booking
6,158,151.0,15.0,Emily White,2025-03-15,7,John Doe,Delivered,New Booking
7,159,152.0,16.0,Frank Miller,2025-03-15,7,John Doe,Delivered,New Booking
8,160,153.0,17.0,Grace Wilson,2025-03-15,7,John Doe,Delivered,New Booking
9,161,154.0,18.0,Hannah Moore,2025-03-15,7,John Doe,Delivered,New Booking


Following code (below) to update the dates in the bookings table to match that of the orders made on 2025-03-21

In [28]:
# Define the bookings that need to be updated to 2025-03-21
bookings_to_update = [
    (195, "2025-03-21", "Admin"),
    (196, "2025-03-21", "Admin"),
    (197, "2025-03-21", "Admin"),
    (198, "2025-03-21", "Admin"),
    (199, "2025-03-21", "Admin"),
]

# Execute UpdateBooking() for each booking
try:
    for booking in bookings_to_update:
        cursor.callproc("UpdateBooking", booking)
    
    # Commit changes
    connection.commit()
    print("\n✅ Bookings successfully updated to 2025-03-21.")

except connector.Error as err:
    print(f"Error updating bookings: {err}")

# Verify the update
updated_bookings_query = """
SELECT * FROM Bookings WHERE BookingID IN (195, 196, 197, 198, 199);
"""

cursor.execute(updated_bookings_query)
df_updated_bookings = pd.DataFrame(cursor.fetchall(), columns=[desc[0] for desc in cursor.description])

# Display updated bookings
print("\n📌 Updated Bookings:")
display(df_updated_bookings)


✅ Bookings successfully updated to 2025-03-21.

📌 Updated Bookings:


,BookingID,CustomerID,BookingDate,TableID,Status
0,195,10,2025-03-21,2,New Booking
1,196,11,2025-03-21,4,New Booking
2,197,12,2025-03-21,6,New Booking
3,198,13,2025-03-21,8,New Booking
4,199,14,2025-03-21,10,New Booking


We are now going to cancel a few orders via cancelling the booking and check the data in orders table and order items table

In [41]:
# Define the Booking ID to cancel
booking_to_cancel = 195  # Change this ID as needed
performed_by = "Alice Johnson"

# Execute CancelBooking()
try:
    cursor.callproc("CancelBooking", (booking_to_cancel, performed_by))
    
    # Commit the transaction
    connection.commit()
    print(f"\nBooking ID {booking_to_cancel} has been successfully cancelled.")

except connector.Error as err:
    print(f"Error cancelling booking: {err}")

# Verify the cancellation
cancelled_booking_query = f"""
SELECT * FROM Bookings WHERE BookingID = {booking_to_cancel};
"""

cursor.execute(cancelled_booking_query)
df_cancelled_booking = pd.DataFrame(cursor.fetchall(), columns=[desc[0] for desc in cursor.description])

# Display the cancelled booking
print("\n📌 Cancelled Booking:")
display(df_cancelled_booking)

Error cancelling booking: 1644 (45000): Error: Booking is already cancelled

📌 Cancelled Booking:


,BookingID,CustomerID,BookingDate,TableID,Status
0,195,10,2025-03-21,2,Cancelled


In [42]:
# Define the bookings to cancel
bookings_to_cancel = [196, 197]
performed_by = "Alice Johnson"

# Execute CancelBooking() for each booking
try:
    for booking_id in bookings_to_cancel:
        cursor.callproc("CancelBooking", (booking_id, performed_by))

    # Commit transaction
    connection.commit()
    print(f"\nBookings {bookings_to_cancel} have been successfully cancelled.")

except connector.Error as err:
    print(f"Error cancelling bookings: {err}")

# Verify the cancellations
cancelled_bookings_query = f"""
SELECT * FROM Bookings WHERE BookingID IN ({", ".join(map(str, bookings_to_cancel))});
"""

cursor.execute(cancelled_bookings_query)
df_cancelled_bookings = pd.DataFrame(cursor.fetchall(), columns=[desc[0] for desc in cursor.description])

# Display the cancelled bookings
print("\n📌 Cancelled Bookings:")
display(df_cancelled_bookings)

Error cancelling bookings: 1644 (45000): Error: Booking is already cancelled

📌 Cancelled Bookings:


,BookingID,CustomerID,BookingDate,TableID,Status
0,196,11,2025-03-21,4,Cancelled
1,197,12,2025-03-21,6,Cancelled


In [43]:
# Retrieve all orders from the Orders table
orders_query = """
SELECT * FROM Orders
WHERE OrderDate = "2025-03-21";
"""

# Execute the query
cursor.execute(orders_query)

# Fetch all records and convert to Pandas DataFrame
df_orders = pd.DataFrame(cursor.fetchall(), columns=[desc[0] for desc in cursor.description])

# Display the orders
print("\n📌 Orders in the Orders Table for 2025-03-21:")
display(df_orders)


📌 Orders in the Orders Table for 2025-03-21:


,OrderID,CustomerID,StaffID,OrderDate,TotalCost,BookingID
0,190,10,20,2025-03-21,82.00,195
1,191,11,20,2025-03-21,69.50,196
2,192,12,20,2025-03-21,25.50,197
3,193,13,20,2025-03-21,25.50,198
4,194,14,20,2025-03-21,24.25,199


At this stage we can see, we have cancelled the order but the procedue didnt account for cancelling the orders in the orders table, for now lets manually do this, we will update proceudre and then test again.

In [44]:
# Define the Order IDs to cancel
orders_to_cancel = [190, 191, 192]
performed_by = "Alice Johnson"

# Cancel orders and remove order items
try:
    for order_id in orders_to_cancel:
        # Remove all items linked to the order
        delete_items_query = f"""
        DELETE FROM Order_Items WHERE OrderID = {order_id};
        """
        cursor.execute(delete_items_query)

        # Update order status to 'Cancelled'
        update_order_query = f"""
        UPDATE Order_Delivery_Status 
        SET Status = 'Cancelled' 
        WHERE OrderID = {order_id};
        """
        cursor.execute(update_order_query)
    
    # Commit the changes
    connection.commit()
    print(f"\nOrders {orders_to_cancel} have been successfully cancelled.")

except connector.Error as err:
    print(f"Error cancelling orders: {err}")

# Verify cancellations in Order_Delivery_Status
cancelled_orders_query = f"""
SELECT * FROM Order_Delivery_Status WHERE OrderID IN ({", ".join(map(str, orders_to_cancel))});
"""
cursor.execute(cancelled_orders_query)
df_cancelled_orders = pd.DataFrame(cursor.fetchall(), columns=[desc[0] for desc in cursor.description])

# Verify orders in the Orders table to see if TotalCost is updated
orders_query = f"""
SELECT * FROM Orders WHERE OrderID IN ({", ".join(map(str, orders_to_cancel))});
"""
cursor.execute(orders_query)
df_orders = pd.DataFrame(cursor.fetchall(), columns=[desc[0] for desc in cursor.description])

# Display the cancelled orders and updated orders table
print("\n📌 Cancelled Orders:")
display(df_cancelled_orders)

print("\n📌 Updated Orders Table:")
display(df_orders)


Orders [190, 191, 192] have been successfully cancelled.

📌 Cancelled Orders:


,DeliveryID,OrderID,Date,Status
0,44,190,2025-03-21,Cancelled
1,45,191,2025-03-21,Cancelled
2,46,192,2025-03-21,Cancelled



📌 Updated Orders Table:


,OrderID,CustomerID,StaffID,OrderDate,TotalCost,BookingID
0,190,10,20,2025-03-21,82.00,195
1,191,11,20,2025-03-21,69.50,196
2,192,12,20,2025-03-21,25.50,197


At this point we can see we need to manually reset these totals in the orders table. Checking the order items table, we can see the items were removed but it did not update the orders table and reset the totalcost to 0, so we need to fix the proceudre.

In [46]:
# Query to fetch all order items
order_items_query = """
SELECT * FROM Order_Items
WHERE OrderID BETWEEN 190 AND 194
"""

cursor.execute(order_items_query)
df_order_items = pd.DataFrame(cursor.fetchall(), columns=[desc[0] for desc in cursor.description])

# Display the order items table
print("\n📌 All Order Items:")
display(df_order_items)



📌 All Order Items:


,OrderItemID,OrderID,MenuID,Quantity,ItemPrice,TotalItemCost
0,243,193,24,1,7.50,7.50
1,244,193,26,2,6.00,12.00
2,245,193,28,1,6.00,6.00
3,246,194,32,2,3.50,7.00
4,247,194,33,1,3.75,3.75
5,248,194,35,1,13.50,13.50


In [47]:
# Define the Order IDs to reset TotalCost
orders_to_reset = [190, 191, 192]

try:
    # Reset TotalCost in the Orders table
    reset_total_cost_query = f"""
    UPDATE Orders 
    SET TotalCost = 0 
    WHERE OrderID IN ({", ".join(map(str, orders_to_reset))});
    """
    cursor.execute(reset_total_cost_query)

    # Commit the changes
    connection.commit()
    print(f"\nTotalCost has been reset for orders {orders_to_reset}.")

except connector.Error as err:
    print(f"Error resetting TotalCost: {err}")

# Verify updated orders table
orders_query = f"""
SELECT * FROM Orders WHERE OrderID IN ({", ".join(map(str, orders_to_reset))});
"""
cursor.execute(orders_query)
df_orders = pd.DataFrame(cursor.fetchall(), columns=[desc[0] for desc in cursor.description])

# Display the updated orders table
print("\n📌 Updated Orders Table:")
display(df_orders)



TotalCost has been reset for orders [190, 191, 192].

📌 Updated Orders Table:


,OrderID,CustomerID,StaffID,OrderDate,TotalCost,BookingID
0,190,10,20,2025-03-21,0.00,195
1,191,11,20,2025-03-21,0.00,196
2,192,12,20,2025-03-21,0.00,197


New test after updating CancelBooking Procedure

In [48]:
# Define the Booking ID to cancel
booking_to_cancel = 198  # Linked to Order ID 193
performed_by = "Alice Johnson"

# Execute CancelBooking() for Booking ID 198
try:
    cursor.callproc("CancelBooking", (booking_to_cancel, performed_by))

    # Commit the transaction
    connection.commit()
    print(f"\n✅ Booking ID {booking_to_cancel} has been successfully cancelled.")

except connector.Error as err:
    print(f"❌ Error cancelling booking: {err}")

# Verify cancellation in the Bookings table
cancelled_booking_query = f"""
SELECT * FROM Bookings WHERE BookingID = {booking_to_cancel};
"""
cursor.execute(cancelled_booking_query)
df_cancelled_booking = pd.DataFrame(cursor.fetchall(), columns=[desc[0] for desc in cursor.description])

# Verify updates in Orders table
orders_query = f"""
SELECT * FROM Orders WHERE BookingID = {booking_to_cancel};
"""
cursor.execute(orders_query)
df_orders = pd.DataFrame(cursor.fetchall(), columns=[desc[0] for desc in cursor.description])

# Verify that order items were removed
order_items_query = f"""
SELECT * FROM Order_Items WHERE OrderID = 193;
"""
cursor.execute(order_items_query)
df_order_items = pd.DataFrame(cursor.fetchall(), columns=[desc[0] for desc in cursor.description])

# Verify cancellation in Order_Delivery_Status
delivery_status_query = f"""
SELECT * FROM Order_Delivery_Status WHERE OrderID = 193;
"""
cursor.execute(delivery_status_query)
df_delivery_status = pd.DataFrame(cursor.fetchall(), columns=[desc[0] for desc in cursor.description])

# Display results
print("\n📌 Cancelled Booking:")
display(df_cancelled_booking)

print("\n📌 Updated Orders Table:")
display(df_orders)

print("\n📌 Order Items After Cancellation (Should be Empty for OrderID 193):")
display(df_order_items)

print("\n📌 Order Delivery Status After Cancellation:")
display(df_delivery_status)


✅ Booking ID 198 has been successfully cancelled.

📌 Cancelled Booking:


,BookingID,CustomerID,BookingDate,TableID,Status
0,198,13,2025-03-21,8,Cancelled



📌 Updated Orders Table:


,OrderID,CustomerID,StaffID,OrderDate,TotalCost,BookingID
0,193,13,20,2025-03-21,0.00,198



📌 Order Items After Cancellation (Should be Empty for OrderID 193):


,OrderItemID,OrderID,MenuID,Quantity,ItemPrice,TotalItemCost



📌 Order Delivery Status After Cancellation:


,DeliveryID,OrderID,Date,Status
0,47,193,2025-03-21,Cancelled


In [13]:
Close cursor and connection
cursor.close()
connection.close()
print("\nConnection closed.")


Connection closed.
